In [ ]:
# Import Dependencies for Scrapping
from bs4 import BeautifulSoup
import requests
import os

# Import Dependencies for data cleasing and SQL load
import pandas as pd
import sqlite3

## Dependencies for GMaps api
import json 
import time
import numpy as np
from IPython.display import clear_output

# Google developer API key
from api_key import g_key

## 1. Scrap Ranking Page

In [ ]:
# Ranking Page
filepath = os.path.join("../html files/World University Rankings - Ranking.html")
with open(filepath, encoding='utf-8') as file:
    htmlRanking = file.read()

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soupRanking = BeautifulSoup(htmlRanking, 'html.parser')

In [ ]:
# Retrieve the anchors for all university names
anchorTitles = soupRanking.find_all('a', class_='ranking-institution-title')

# Create empty list
universitiesNames = []

# loop over results to get the data
for title in anchorTitles:
    name = title.text
    universitiesNames.append(name)

In [ ]:
# Retrieve the parent divs for locations
anchorLocations = soupRanking.find_all('div', class_='location')

# Create empty list
universitiesLocations = []

# Loop over results to get the data
for locations in anchorLocations:
    country = locations.find('a').text
    universitiesLocations.append(country)

In [ ]:
# Retrieve the rows for get stats - Number of Students
trRole = soupRanking.find_all('td', class_='stats_number_students')

# Create empty list
numberStudents = []

# Loop over results to get the data
for tr in trRole:
    number = tr.text
    numberStudents.append(number)

In [ ]:
# Retrieve the rows for get stats - Student Staff Ratio
trRole = soupRanking.find_all('td', class_='stats stats_student_staff_ratio')

# Create empty list
studentsStaffRatio = []

# Loop over results to get the data
for tr in trRole:
    ratio = tr.text
    studentsStaffRatio.append(ratio)

In [ ]:
# Retrieve the rows for get stats - % of international students
trRole = soupRanking.find_all('td', class_='stats stats_pc_intl_students')

# Create empty list
percIntStudents = []

# Loop over results to get the data
for tr in trRole:
    perc = tr.text
    percIntStudents.append(perc)

In [ ]:
# Retrieve the rows for get stats - female/male ratio
trRole = soupRanking.find_all('td', class_='stats stats_female_male_ratio')

# Create empty list
genderRatio = []

# Loop over results to get the data
for tr in trRole:
    ratio = tr.text
    genderRatio.append(ratio)

In [ ]:
# Create dictionary

rankingTable = {
    'ranking' : range(1, len(universitiesNames)+1),
    'title' : universitiesNames,
    'location' : universitiesLocations,
    'number_students' : numberStudents,
    'students_staff_ratio' : studentsStaffRatio,
    'perc_intl_students' : percIntStudents,
    'gender_ratio' : genderRatio
}

In [ ]:
# Transform dict to dataframe
ranking_df = pd.DataFrame.from_dict(rankingTable)
ranking_df

In [ ]:
# To CSV
ranking_df.to_csv('../csv files/universities_ranking.csv', header=True, encoding='utf-8-sig')

## 2. Scrap Scores Page

In [ ]:
# Scores Page
filepath = os.path.join("../html files/World University Rankings - Scores.html")
with open(filepath, encoding='utf-8') as file:
    htmlScores = file.read()

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soupScores = BeautifulSoup(htmlScores, 'html.parser')

In [ ]:
# Retrieve the anchors for all university names
anchorTitles = soupScores.find_all('a', class_='ranking-institution-title')

# Create empty list
universitiesNames = []

# loop over results to get the data
for title in anchorTitles:
    name = title.text
    universitiesNames.append(name)

In [ ]:
# Retrieve the parent divs for locations
anchorLocations = soupScores.find_all('div', class_='location')

# Create empty list
universitiesLocations = []

# Loop over results to get the data
for locations in anchorLocations:
    country = locations.find('a').text
    universitiesLocations.append(country)

In [ ]:
# Retrieve the rows for get stats - Overall Score
trRole = soupScores.find_all('td', class_='scores overall-score')

# Create empty list
overallScore = []

# Loop over results to get the data
for tr in trRole:
    score = tr.text
    overallScore.append(score)

In [ ]:
# Retrieve the rows for get stats - Teaching Score
trRole = soupScores.find_all('td', class_='scores teaching-score')

# Create empty list
teachingScore = []

# Loop over results to get the data
for tr in trRole:
    score = tr.text
    teachingScore.append(score)

In [ ]:
# Retrieve the rows for get stats - Research Score
trRole = soupScores.find_all('td', class_='scores research-score')

# Create empty list
researchScore = []

# Loop over results to get the data
for tr in trRole:
    score = tr.text
    researchScore.append(score)

In [ ]:
# Retrieve the rows for get stats - Citations Score
trRole = soupScores.find_all('td', class_='scores citations-score')

# Create empty list
citationsScore = []

# Loop over results to get the data
for tr in trRole:
    score = tr.text
    citationsScore.append(score)

In [ ]:
# Retrieve the rows for get stats - Industry Income Score
trRole = soupScores.find_all('td', class_='scores industry_income-score')

# Create empty list
industryIncomeScore = []

# Loop over results to get the data
for tr in trRole:
    score = tr.text
    industryIncomeScore.append(score)

In [ ]:
# Retrieve the rows for get stats - International Outlook Score
trRole = soupScores.find_all('td', class_='scores international_outlook-score')

# Create empty list
intlOutlookScore = []

# Loop over results to get the data
for tr in trRole:
    score = tr.text
    intlOutlookScore.append(score)

In [ ]:
# Create dictionary
scoreTable = {
    'ranking' : range(1, len(universitiesNames)+1),
    'title' : universitiesNames,
    'location' : universitiesLocations,
    'overall_score' : overallScore,
    'teaching_score' : teachingScore,
    'research_score' : researchScore,
    'citations_score' : citationsScore,
    'industry_income_score' : industryIncomeScore,
    'intl_outlook_score' : intlOutlookScore
}

In [ ]:
# Transform dict to dataframe
score_df = pd.DataFrame.from_dict(scoreTable)
score_df

In [ ]:
# To CSV
score_df.to_csv('../csv files/universities_score.csv', header=True, encoding='utf-8-sig')

## 3. Merge

In [ ]:
# Concatenate Dataframes
universities_df = pd.merge(ranking_df, score_df, on='ranking', how='inner')
universities_df.drop(['title_y', 'location_y'], axis=1, inplace=True)
universities_df.rename(columns={'title_x' : 'title', 'location_x' : 'location'}, inplace=True)
universities_df['number_students'] = universities_df['number_students'].str.replace(',', '').astype(float)
universities_df

## 4. Use Geocode API to look for Lat/Long/Continent

In [ ]:
# To do so, first we need to add new columns to universities_df, where info is going to be stored
# Add columns for lat, lng, continent
# Note that we used "" to specify initial entry
universities_df["latitude"] = ""
universities_df["longitude"] = ""
universities_df.head()

In [ ]:
# Target university
target_university = "University of Brighton|United Kingdom"


# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_university, g_key)

# print('---')
# print(target_url)
# print('---')

geo_data = requests.get(target_url).json()

# Print the json
print(geo_data)

In [ ]:
# Test the entries for the JSON
lat = geo_data["results"][0]["geometry"]["location"]["lat"]
lng = geo_data["results"][0]["geometry"]["location"]["lng"]

print(lat, lng)

In [ ]:
# Let's make it recursive
count_ok = 0
count_nok = 0
total_universities = len(universities_df["ranking"])

for index, row in universities_df.iterrows():
    # get the name of the university
    target_university = row["title"] + "|" + row["location"]

    # Use the search term: "International Airport" and our lat/lng
    base_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_university, g_key)

    # make request
    geo_data = requests.get(base_url)
    

    # convert to json
    geo_data = geo_data.json()
    
    clear_output(wait=True)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        #Lets print a mesage
        print(f'Iteration {count_ok + count_nok +1} of {total_universities} - running . . .')
        print(f'Looking for', target_university, '...')
        
        universities_df.loc[index, "latitude"] = geo_data["results"][0]["geometry"]["location"]["lat"]
        universities_df.loc[index, "longitude"] = geo_data["results"][0]["geometry"]["location"]["lng"]
        time.sleep(1)
        
        count_ok += 1
        
    except (KeyError, IndexError):
        #Lets print a mesage
        print("Missing field/result... skipping.")
        
        universities_df.loc[index, "latitude"] = universities_df.loc[index, 'latitude'] = np.nan
        universities_df.loc[index, "longitude"] = universities_df.loc[index, 'longitude'] = np.nan
        
        count_nok += 1
        
    test = test + 1
       
print("------------")
print('We are Done!')
print(f"We've searched for {total_universities} universities and we've got \n{count_ok} successful responses and \n{count_nok} unsuccessful responses")
print("------------")

In [ ]:
universities_df

## 6. Append Continent info

In [ ]:
continent_df = pd.read_csv("../csv files/universities_continent.csv")
continent_df.head()

In [ ]:
# Concatenate Dataframes
universities_full_df = pd.merge(continent_df, universities_df, on='ranking', how='inner')
universities_full_df.drop(['title_y', 'location_y'], axis=1, inplace=True)
universities_full_df.rename(columns={'title_x' : 'title', 'location_x' : 'location'}, inplace=True)
universities_full_df

## 7. Generate Final CSV File

In [ ]:
universities_full_df= universities_full_df.astype(
    {
        "number_students": float, 
        "students_staff_ratio": float,
        "overall_score": str,        
        "teaching_score": float,       
        "research_score" : float,      
        "citations_score"  : float,    
        "industry_income_score": float,
        "intl_outlook_score"  : float,
        "latitude" : float,
        "longitude" : float
    }
)


In [ ]:
universities_full_df.dtypes

In [ ]:
# To CSV
universities_full_df.to_csv('../csv files/universities_rank_score.csv', header=True, encoding='utf-8-sig')

## 6. Creating the SQLite

In [ ]:
conn = sqlite3.connect('../universities.sqlite')
c = conn.cursor()

In [ ]:
c.execute("DROP TABLE IF EXISTS universities_ranking")
conn.commit()

In [ ]:
c.execute("DROP TABLE IF EXISTS pd_universities_ranking")
conn.commit()

In [ ]:
# Create table
c.execute(
    """
    CREATE TABLE universities_ranking (
    ranking INTEGER NOT NULL PRIMARY KEY,
    title TEXT,
    location TEXT,
    continent TEXT,
    number_students REAL,
    students_staff_ratio TEXT,
    perc_intl_students TEXT,
    gender_ratio TEXT,
    overall_score REAL,
    teaching_score REAL,
    research_score REAL,
    citations_score REAL,
    industry_income_score REAL,
    intl_outlook_score REAL,
    latitude REAL,
    longitude REAL
    )""")

conn.commit()

In [ ]:
# Create table
c.execute(
    """
    CREATE TABLE pd_universities_ranking (
    ranking INTEGER NOT NULL PRIMARY KEY,
    title TEXT,
    location TEXT,
    continent TEXT,
    number_students REAL,
    students_staff_ratio TEXT,
    perc_intl_students TEXT,
    gender_ratio TEXT,
    overall_score REAL,
    teaching_score REAL,
    research_score REAL,
    citations_score REAL,
    industry_income_score REAL,
    intl_outlook_score REAL,
    latitude REAL,
    longitude REAL
    )""")

conn.commit()

In [ ]:
universities_full_df.to_sql('pd_universities_ranking', conn, if_exists='replace', index = False)

In [ ]:
c.execute("""
INSERT INTO universities_ranking SELECT * FROM pd_universities_ranking;
""")
conn.commit()

In [ ]:
c.execute("""
DROP TABLE pd_universities_ranking;
""")
conn.commit()

In [ ]:
c.execute("SELECT * FROM universities_ranking")

In [ ]:
print(*c.fetchmany(10), sep = "\n")

In [ ]:
conn.close()